In [ ]:
from libs.extract import ExtractFeatures
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


webshell_data = []
benign_data = []

webshells = os.listdir(os.path.curdir + '/data/webshells')
random.shuffle(webshells)

benigns = os.listdir(os.path.curdir + '/data/benigns')
random.shuffle(benigns)

webshellsLongest = []
benignsLongest = []
webshellsEntropy = []
benignsEntropy = []

for file in webshells[0:500]:
    extractor = ExtractFeatures('/data/webshells/'+file, 'dataset.csv')
    result = extractor.extract_function_names()
    webshellsLongest.append({ 'malware': 1, 'longest': extractor.extract_longest_string()})
    webshellsEntropy.append({ 'malware': 1, 'entropy': extractor.extract_entropy_file()})
    result['class'] = 'malware'
    webshell_data.append(result)
  
for file in benigns[0:500]:
    extractor = ExtractFeatures('/data/benigns/'+file, 'dataset.csv')
    result = extractor.extract_function_names()
    benignsLongest.append({ 'malware': 0, 'longest': extractor.extract_longest_string()})
    benignsEntropy.append({ 'malware': 0, 'entropy': extractor.extract_entropy_file()})
    result['class'] = 'benign'
    benign_data.append(result)

df_webshell = pd.DataFrame(webshell_data).fillna(0)
df_benigns = pd.DataFrame(benign_data).fillna(0)

# dataset = pd.concat([df_webshell, df_benigns]).fillna(0)

# col = dataset.pop('class')
# dataset.insert(0, 'class', col)
# dataset.sample(frac=1).reset_index(drop=True).to_csv('dataset.csv')

df = pd.DataFrame(webshellsLongest + benignsLongest)
dataset = df.sample(frac=1).reset_index(drop=True)

df_2 = pd.DataFrame(webshellsEntropy + benignsEntropy)
dataset_2 = df_2.sample(frac=1).reset_index(drop=True)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


df_webshell = pd.DataFrame(webshellsLongest)
df_benign = pd.DataFrame(benignsLongest)

plt.plot(df_webshell['longest'], c='red')
plt.plot(df_benign['longest'])

plt.legend(["malware", "benign"], loc ="right")
plt.show()


df_webshell_entropy = pd.DataFrame(webshellsEntropy)
df_benign_entropy = pd.DataFrame(benignsEntropy)


print ("Avergate of Webshell %f and Benign is %f" % (df_webshell_entropy['entropy'].mean(), df_benign_entropy['entropy'].mean()))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


df_webshell = pd.DataFrame(webshellsEntropy)
df_benign = pd.DataFrame(benignsEntropy)

xpoints = np.array(df_benign.index.array)
ypoints = np.array(df_benign['entropy'])

plt.scatter(xpoints, ypoints)

xpoints = np.array(df_webshell.index.array)
ypoints = np.array(df_webshell['entropy'])

plt.scatter(xpoints, ypoints, c='red')
plt.legend(['benign', 'malware'])

plt.show()



In [ ]:
from libs.extract import ExtractFeatures
import os
import random
import pandas as pd

file = '5284c689364b3d94f6abf428e87b35c6.php'
extractor = ExtractFeatures('/data/webshells/'+file, 'dataset_test.csv')
extractor.extract_longest_string()


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt



df = pd.read_csv('./dataset.csv')
X = df.drop(columns=['class', 'no']).to_numpy()
Y = df['class'].to_numpy()

std_scaler = StandardScaler()
scaled_df = std_scaler.fit_transform(X)

pca = PCA(n_components=10)
new_X = pca.fit_transform(X)


# for label in set(Y):
#     X_class = new_X[Y == label]
#     plt.scatter(X_class[:, 0], X_class[:, 1], label=label)

# plt.legend()


# plt.bar(
#     range(1,len(pca.explained_variance_)+1),
#     pca.explained_variance_
#     )
 
 
# plt.xlabel('PCA Feature')
# plt.ylabel('Explained variance')
# plt.title('Feature Explained Variance')
# plt.show()


# pca_features = pca.fit_transform(scaled_df)

# pca_df = pd.DataFrame(data=pca_features)
# pca_df




In [51]:
import pandas as pd
import plotly.express as px
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

df = pd.read_csv('./dataset.csv')
features = df.drop(columns=['no', 'class']).columns
df['class'] = le.fit_transform(df['class'])


pca = PCA()
components = pca.fit_transform(df[features[0:5]])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(5),
    color=df["class"]
)
fig.update_traces(diagonal_visible=False)
fig.show()


In [32]:
import plotly.express as px
from sklearn.decomposition import PCA

df = px.data.iris()
features = ["sepal_width", "sepal_length", "petal_width", "petal_length"]

pca = PCA()
components = pca.fit_transform(df[features])
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=df["species"]
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [11]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


le = preprocessing.LabelEncoder()

df = pd.read_csv('./dataset.csv')

target = le.fit_transform(df['class'])
train = df.drop(columns=['no', 'class']).values

X_std = StandardScaler().fit_transform(train)
# Calculating Eigenvectors and eigenvalues of Cov matirx
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenvalue, eigenvector pair from high to low
eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance



In [14]:
from sklearn.decomposition import PCA
[ n for n,i in enumerate(cum_var_exp) if i>90 ][0]

pca = PCA(n_components=274)
pca.fit(X_std)
X_228d = pca.transform(X_std)
X_228d.shape

(1000, 274)